# Description

It splits gene expression data from GTEx v8 by tissue and saves a gene id/symbol mapping file.

# Modules

In [1]:
import pandas as pd
from tqdm import tqdm

from ccc.utils import simplify_string
from ccc import conf

# Settings

In [2]:
OUTPUT_DIR = conf.GTEX["DATA_DIR"] / "data_by_tissue"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/data/gtex_v8/data_by_tissue')

# Data loading

## GTEx v8

### Sample metadata

In [3]:
gtex_sample_attrs = pd.read_csv(
    conf.GTEX["SAMPLE_ATTRS_FILE"],
    sep="\t",
)

In [4]:
gtex_sample_attrs.shape

(22951, 63)

In [5]:
gtex_sample_attrs.head()

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
0,GTEX-1117F-0003-SM-58Q7G,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GTEX-1117F-0003-SM-5DWSB,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GTEX-1117F-0003-SM-6WBT7,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GTEX-1117F-0011-R10a-SM-AHZ7F,NaN,"B1, A1",NaN,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,1193.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GTEX-1117F-0011-R10b-SM-CYKQ8,NaN,"B1, A1",NaN,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Get tissues names

In [6]:
gtex_tissues = gtex_sample_attrs["SMTSD"].unique()
display(len(gtex_tissues))
display(gtex_tissues)

55

array(['Whole Blood', 'Brain - Frontal Cortex (BA9)',
       'Adipose - Subcutaneous', 'Muscle - Skeletal', 'Artery - Tibial',
       'Artery - Coronary', 'Heart - Atrial Appendage',
       'Adipose - Visceral (Omentum)', 'Ovary', 'Uterus', 'Vagina',
       'Breast - Mammary Tissue', 'Skin - Not Sun Exposed (Suprapubic)',
       'Minor Salivary Gland', 'Brain - Cortex', 'Adrenal Gland',
       'Thyroid', 'Lung', 'Spleen', 'Pancreas', 'Esophagus - Muscularis',
       'Esophagus - Mucosa', 'Esophagus - Gastroesophageal Junction',
       'Stomach', 'Colon - Sigmoid', 'Small Intestine - Terminal Ileum',
       'Colon - Transverse', 'Prostate', 'Testis',
       'Skin - Sun Exposed (Lower leg)', 'Nerve - Tibial',
       'Heart - Left Ventricle', 'Pituitary', 'Brain - Cerebellum',
       'Cells - Cultured fibroblasts', 'Artery - Aorta',
       'Cells - EBV-transformed lymphocytes',
       'Brain - Cerebellar Hemisphere', 'Brain - Caudate (basal ganglia)',
       'Brain - Nucleus accumbens (ba

# Get sample IDs for each tissue

In [7]:
# first, get all sample IDs with expression data
gtex_all_sample_ids_with_expr_data = set(
    pd.read_csv(
        conf.GTEX["DATA_TPM_GCT_FILE"],
        sep="\t",
        skiprows=2,
        nrows=1,
        usecols=lambda x: x not in ("Name", "Description"),
    ).columns
)

In [8]:
len(gtex_all_sample_ids_with_expr_data)

17382

In [9]:
list(gtex_all_sample_ids_with_expr_data)[:10]

['GTEX-1269C-2626-SM-5EQ5C',
 'GTEX-1RMOY-1126-SM-EV79E',
 'GTEX-13IVO-0726-SM-5LZYV',
 'GTEX-1RLM8-0011-R2a-SM-EZ6N4',
 'GTEX-1GF9V-0126-SM-9WYT2',
 'GTEX-1HSKV-2926-SM-B2LWJ',
 'GTEX-1212Z-2626-SM-5YY8P',
 'GTEX-14BIM-0011-R5b-SM-5S2RM',
 'GTEX-14JG6-2126-SM-6EU2F',
 'GTEX-14BMU-2426-SM-5RQIJ']

In [10]:
# get sample IDs by tissue
sample_ids_by_tissue = {
    tissue_name: sorted(
        list(
            gtex_all_sample_ids_with_expr_data.intersection(
                set(
                    gtex_sample_attrs[gtex_sample_attrs["SMTSD"] == tissue_name][
                        "SAMPID"
                    ].tolist()
                )
            )
        )
    )
    for tissue_name in gtex_tissues
}

In [11]:
assert len(gtex_tissues) == len(sample_ids_by_tissue)

In [12]:
sample_ids_by_tissue["Whole Blood"][:10]

['GTEX-111YS-0006-SM-5NQBE',
 'GTEX-1122O-0005-SM-5O99J',
 'GTEX-1128S-0005-SM-5P9HI',
 'GTEX-113IC-0006-SM-5NQ9C',
 'GTEX-113JC-0006-SM-5O997',
 'GTEX-117XS-0005-SM-5PNU6',
 'GTEX-117YW-0005-SM-5NQ8Z',
 'GTEX-1192W-0005-SM-5NQBQ',
 'GTEX-1192X-0005-SM-5NQC3',
 'GTEX-11DXW-0006-SM-5NQ7Y']

In [13]:
# all IDs are unique
assert all(
    [
        len(sample_ids_by_tissue[tissue_name])
        == len(set(sample_ids_by_tissue[tissue_name]))
        for tissue_name in sample_ids_by_tissue.keys()
    ]
)

## Show sample size by tissue

In [14]:
tissue_sample_size = pd.DataFrame(
    [{"tissue": k, "sample_size": len(v)} for k, v in sample_ids_by_tissue.items()]
)

In [15]:
tissue_sample_size = tissue_sample_size.sort_values("sample_size", ascending=False)
display(tissue_sample_size)

,tissue,sample_size
3,Muscle - Skeletal,803
0,Whole Blood,755
29,Skin - Sun Exposed (Lower leg),701
2,Adipose - Subcutaneous,663
4,Artery - Tibial,663
16,Thyroid,653
30,Nerve - Tibial,619
12,Skin - Not Sun Exposed (Suprapubic),604
17,Lung,578
21,Esophagus - Mucosa,555


In [16]:
# some testing
_tmp = tissue_sample_size.set_index("tissue").squeeze()
assert _tmp.loc["Muscle - Skeletal"] == 803
assert _tmp.loc["Whole Blood"] == 755
assert _tmp.loc["Skin - Not Sun Exposed (Suprapubic)"] == 604
assert _tmp.loc["Kidney - Medulla"] == 4

These numbers match those you can find here: https://gtexportal.org/home/tissueSummaryPage#sampleCountsPerTissue

# Split expression data by tissue

In [17]:
pbar = tqdm(tissue_sample_size["tissue"])

gene_id_symbol_map_tuples = set()

for tissue_name in pbar:
    pbar.set_description(tissue_name)

    tissue_ids = sample_ids_by_tissue[tissue_name]
    if len(tissue_ids) == 0:
        continue

    tissue_data = pd.read_csv(
        conf.GTEX["DATA_TPM_GCT_FILE"],
        sep="\t",
        skiprows=2,
        usecols=["Name", "Description"] + tissue_ids,
    )

    tissue_data = tissue_data.rename(
        columns={
            "Name": "gene_ens_id",
            "Description": "gene_symbol",
        }
    )

    # add gene id / gene symbol to mapping variable
    gene_id_symbol_map_tuples.update(
        tissue_data[["gene_ens_id", "gene_symbol"]].itertuples(index=False)
    )

    tissue_data = tissue_data.drop(columns=["gene_symbol"]).set_index("gene_ens_id")

    assert not tissue_data.isna().any().any()
    assert tissue_data.index.is_unique
    assert tissue_data.columns.is_unique

    # save
    tissue_name_simple = simplify_string(simplify_string(tissue_name.lower()))
    tissue_data.to_pickle(path=OUTPUT_DIR / f"gtex_v8_data_{tissue_name_simple}.pkl")

Cells - Leukemia cell line (CML): 100%|██████████| 55/55 [47:47<00:00, 52.14s/it]


## Testing

Here I take a random tissue file that was just generate it, read it, and check the expected number of samples, and that the expected sample IDs are there. I also check that same expression values for gene/sample pairs are correct (I read those from a different console, just to make sure I'm not making a mistake here).

In [18]:
_tmp = pd.read_pickle(OUTPUT_DIR / "gtex_v8_data_brain_cerebellar_hemisphere.pkl")

In [19]:
# taken from GTEx webpage (see above)
assert _tmp.shape[1] == 215

In [20]:
assert "GTEX-11DXY-0011-R11a-SM-DNZZN" in _tmp.columns
assert "GTEX-WL46-0011-R11A-SM-3MJFT" in _tmp.columns
assert "GTEX-ZF28-0011-R11a-SM-4WWEI" in _tmp.columns

In [21]:
_v = _tmp.loc["ENSG00000223972.5", "GTEX-11DXY-0011-R11a-SM-DNZZN"]
assert _v == 0.04045, _v
_v = _tmp.loc["ENSG00000278267.1", "GTEX-11DXY-0011-R11a-SM-DNZZN"]
assert _v == 0.0, _v

_v = _tmp.loc["ENSG00000233327.10", "GTEX-WL46-0011-R11A-SM-3MJFT"]
assert _v == 146.4000, _v
_v = _tmp.loc["ENSG00000237118.2", "GTEX-WL46-0011-R11A-SM-3MJFT"]
assert _v == 0.3357, _v

_v = _tmp.loc["ENSG00000233327.10", "GTEX-ZF28-0011-R11a-SM-4WWEI"]
assert _v == 30.7200, _v
_v = _tmp.loc["ENSG00000186907.7", "GTEX-ZF28-0011-R11a-SM-4WWEI"]
assert _v == 0.94720, _v

# Save gene mappings

In [22]:
list(gene_id_symbol_map_tuples)[:5]

[Pandas(gene_ens_id='ENSG00000222320.1', gene_symbol='RNU6-1050P'),
 Pandas(gene_ens_id='ENSG00000200554.1', gene_symbol='RNU6-1020P'),
 Pandas(gene_ens_id='ENSG00000115944.14', gene_symbol='COX7A2L'),
 Pandas(gene_ens_id='ENSG00000254158.1', gene_symbol='RP11-281O15.5'),
 Pandas(gene_ens_id='ENSG00000254101.5', gene_symbol='LINC02055')]

In [23]:
gene_mappings = pd.DataFrame(gene_id_symbol_map_tuples)

In [24]:
gene_mappings.shape

(56200, 2)

In [25]:
gene_mappings.head()

,gene_ens_id,gene_symbol
0,ENSG00000222320.1,RNU6-1050P
1,ENSG00000200554.1,RNU6-1020P
2,ENSG00000115944.14,COX7A2L
3,ENSG00000254158.1,RP11-281O15.5
4,ENSG00000254101.5,LINC02055


## Save

In [26]:
output_filename = conf.GTEX["DATA_DIR"] / "gtex_gene_id_symbol_mappings.pkl"
display(output_filename)

PosixPath('/opt/data/data/gtex_v8/gtex_gene_id_symbol_mappings.pkl')

In [27]:
gene_mappings.to_pickle(output_filename)

## Testing

In [28]:
gene_mappings = pd.read_pickle(output_filename)

In [29]:
# no null
assert gene_mappings.dropna(how="any").shape == gene_mappings.shape

In [30]:
# no duplicates
assert gene_mappings.drop_duplicates().shape == gene_mappings.shape

In [31]:
# check gene id and gene symbol lengths (check no empty entries)
_tmp = gene_mappings.copy()
_tmp = _tmp.assign(id_len=gene_mappings["gene_ens_id"].apply(len))
_tmp = _tmp.assign(symbol_len=gene_mappings["gene_symbol"].apply(len))

In [32]:
_tmp_unique = _tmp["id_len"].unique()
display(_tmp_unique)

array([17, 18, 23, 24])

In [33]:
_tmp.drop_duplicates(subset=["id_len"])

,gene_ens_id,gene_symbol,id_len,symbol_len
0,ENSG00000222320.1,RNU6-1050P,17,10
2,ENSG00000115944.14,COX7A2L,18,7
1432,ENSG00000229232.6_PAR_Y,KRT18P53,23,8
3780,ENSG00000182378.13_PAR_Y,PLCXD1,24,6


Unique gene id lengths seem to be valid

In [34]:
assert set(_tmp_unique) == set([17, 18, 24, 23])

In [35]:
_tmp_unique = _tmp["symbol_len"].unique()
display(_tmp_unique)

array([10,  7, 13,  9,  5,  6, 12,  8,  4,  3, 11, 15, 14, 19,  2, 18, 17,
       16,  1])

No gene symbol is empty, that's good

In [36]:
assert (_tmp_unique > 0).all()

In [37]:
assert _tmp_unique.min() == 1
assert _tmp_unique.max() == 19

In [38]:
# show how different gene symbol's lengths look like
_tmp.drop_duplicates(subset=["symbol_len"]).sort_values("symbol_len")

,gene_ens_id,gene_symbol,id_len,symbol_len
6363,ENSG00000164458.9,T,17,1
633,ENSG00000201368.2,U3,17,2
39,ENSG00000109072.13,VTN,18,3
38,ENSG00000138768.14,USO1,18,4
8,ENSG00000184730.10,APOBR,18,5
9,ENSG00000232991.5,GACAT1,17,6
2,ENSG00000115944.14,COX7A2L,18,7
17,ENSG00000271638.1,BNIP3P19,17,8
4,ENSG00000254101.5,LINC02055,17,9
0,ENSG00000222320.1,RNU6-1050P,17,10


Unique gene symbol lengths seem to be valid

In [39]:
assert gene_mappings["gene_ens_id"].unique().shape[0] == gene_mappings.shape[0]

In [40]:
# some gene symbols map to multiple gene ids
display(gene_mappings["gene_symbol"].unique().shape)
assert gene_mappings["gene_symbol"].unique().shape[0] < gene_mappings.shape[0]

(54592,)

In [41]:
# show some duplicated gene symbols
gene_mappings[gene_mappings["gene_symbol"].duplicated(keep=False)].sort_values(
    "gene_symbol"
)

,gene_ens_id,gene_symbol
56122,ENSG00000275305.1,5S_rRNA
49672,ENSG00000274097.1,5S_rRNA
49571,ENSG00000274059.1,5S_rRNA
4303,ENSG00000277411.1,5S_rRNA
32225,ENSG00000212595.1,5S_rRNA
...,...,...
52200,ENSG00000275939.1,uc_338
2260,ENSG00000274845.1,uc_338
881,ENSG00000278236.1,uc_338
17000,ENSG00000278218.1,uc_338


In [42]:
_tmp = gene_mappings.set_index("gene_ens_id").squeeze()

In [43]:
assert _tmp.loc["ENSG00000223972.5"] == "DDX11L1"
assert _tmp.loc["ENSG00000243485.5"] == "MIR1302-2HG"
assert _tmp.loc["ENSG00000274059.1"] == "5S_rRNA"  # repeated gene
assert _tmp.loc["ENSG00000275305.1"] == "5S_rRNA"  # repeated gene